In [4]:
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
import datetime
import os

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
source_path = 'datalake/cleaned'
target_path = 'datalake/enriched'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_store_products = (
    spark
    .read
    .parquet(f"./jupyter_workspace/{source_path}/store_products-1-1-/*")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_enriched_products = (
    df_store_products
    .select(
        df_store_products.ProductID,
        df_store_products.ProductDesc,
        df_store_products.ProductNumber,
        df_store_products.MakeFlag,
        F.when(df_store_products.Color.isNull(), F.lit('N/A')).otherwise(df_store_products.Color).alias('Color'),
        df_store_products.SafetyStockLevel,
        df_store_products.ReorderPoint,
        df_store_products.StandardCost,
        df_store_products.ListPrice,
        df_store_products.Size,
        df_store_products.SizeUnitMeasureCode,
        df_store_products.Weight,
        df_store_products.WeightUnitMeasureCode,        
        F.when(df_store_products.ProductCategoryName.isNull() & df_store_products.ProductSubCategoryName.isin('Gloves', 'Shorts', 'Socks', 'Tights', 'Vests'), F.lit('Clothing'))
         .when(df_store_products.ProductCategoryName.isNull() & df_store_products.ProductSubCategoryName.isin('Locks', 'Lights', 'Headsets', 'Helmets', 'Pedals', 'Pumps'), F.lit('Accessories'))
         .when(df_store_products.ProductCategoryName.isNull() & df_store_products.ProductSubCategoryName.isin('Wheels', 'Saddles') | df_store_products.ProductSubCategoryName.contains('Frames'), F.lit('Components'))
         .otherwise(df_store_products.ProductCategoryName).alias('ProductCategoryName'),        
        df_store_products.ProductSubCategoryName
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
(
    df_enriched_products
    .coalesce(1)
    .write
    .mode('overwrite')
    .parquet(f"./jupyter_workspace/{target_path}/publish_product/")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…